# Machine learning headline analysis to predict daily stock movement
## Goal
- When should we buy and sell stocks for profit

## Stratergy
- Load comment data from reddit
- create sentimate label for each day
- merge with market data
- run classfication and evaluation

## Datasets
- https://www.kaggle.com/aceofit/stockmarketdatafrom1996to2020
- https://www.kaggle.com/theriley106/wallstreetbetscomments?select=wsbData.json

## Data Preprocessing

In [2]:
#Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Sentimate
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm

#Machine learning
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Evaluation
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
wsb = pd.read_csv('wallstreetbets_comments.csv')

C:\Users\Phoebe\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,5,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
#upvotes, awards, and number of replies could also be valuable here in the future
wsb_df = wsb[['body', 'timestamp', 'score']]

In [7]:
#this should be done in parallel
analyser = SentimentIntensityAnalyzer()

errors = 0
scores=[]
for item in tqdm(wsb_df['body']):
    sentiment_score=0
    try:
        sentiment_score=sentiment_score+analyser.polarity_scores(item)['compound']
    except TypeError:
        errors = errors+1
        sentiment_score=0
    
    scores.append(sentiment_score)
    
wsb_df['sentiment_score'] = scores
print(errors)

100%|██████████| 13476652/13476652 [34:41<00:00, 6475.28it/s] 


10625


<ipython-input-7-f3a55ca3f85b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wsb_df['sentiment_score'] = scores


In [46]:
wsb_df.rename(columns={'timestamp': 'Date'}, inplace=True)
pd.to_datetime(wsb_df['Date']).dt

In [60]:
pd.to_datetime(wsb_df['Date']).dt.normalize()

<ipython-input-60-6c0e897d29ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wsb_df2['Date'] = pd.to_datetime(wsb_df['Date']).dt.normalize()


In [67]:
tickers = pd.read_excel('Tickers.xlsx')

#filtered_tickers = tickers[tickers['Country'] == 'Australia']
filtered_tickers = tickers

count = 0

best_score = 0

#this should be done in parallel

for ticker in filtered_tickers['Ticker']:
    #1. process stock
    stock = pd.read_csv('Data/Data/'+ticker+'/'+ticker+'.csv')
    
    if stock.empty:
        continue
    
    
    stock = stock.dropna()
    
    if stock.shape[0] < 100:
        continue
    
    #format
    stock['Date'] = pd.to_datetime(stock['Date'])
    
    #attach label
    stock['Label'] = (stock['Close'] < stock.shift(periods=-1)['Open']).astype(int)
    stock.drop(stock.tail(1).index,inplace=True)
    
    stock_label_count = stock["Label"].value_counts()
    
    #ignor unblanaced stocks
    if (abs(stock_label_count[0] - stock_label_count[1]) / stock_label_count[0]) > 0.2:
        continue
        
    #attach features for each day or drop days with missing features
    df = pd.merge(wsb_df, stock[['Date','Label']], on='Date')
    
    X_train, X_test, y_train, y_test = train_test_split(
        df['sentiment_score'],
        df['Label'],
        random_state = 42,
        stratify = df['Label'])
    
    model = MultinomialNB(alpha=0.1)

    model.fit(X_train, y_train)
    
    model_predict = model.predict(X_test)
    
    #evaluate(model_predict, y_test)
    score = metrics.balanced_accuracy_score(y_test, model_predict)
    
    if best_score < score:
        best_score = score
        
    print(count)
    
    count = count+1
    
    if count > 0:
        break

ValueError: Expected 2D array, got 1D array instead:
array=[-0.3818  0.5994  0.4404 ...  0.     -0.4404 -0.1027].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Hyper parameter tuning

In [ ]:
"""
X_train, X_test, y_train, y_test = train_test_split(
        df['headline_text'],
        df['Label'],
        random_state = 42,
        stratify = df['Label'])

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

grid_search = GridSearchCV(text_clf, parameters, cv=10, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

print('Best Score: ', grid_search.best_score_) 
print('Best Params: ', grid_search.best_params_)


loss = ['hinge', 'log', 'modified_huber', 'squared_hinge','perceptron']
penalty = ['l1', 'l2', 'elasticnet'] 
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000] 
learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive'] 
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
eta0 = [1, 10, 100] 

param_distributions = dict(loss=loss,
                           penalty=penalty,
                           alpha=alpha, 
                           learning_rate=learning_rate, 
                           class_weight=class_weight, 
                           eta0=eta0) 

random = RandomizedSearchCV(estimator=sgd,
                            param_distributions=param_distributions,
                            scoring='roc_auc',
                            verbose=1, n_jobs=-1, 
                            n_iter=100) 

random.fit(X_train, y_train)

print('Best Score: ', random.best_score_) 
print('Best Params: ', random.best_params_)
"""